In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser

### Scarpe news info from NASA website

In [ ]:
url_nasa = "https://mars.nasa.gov/news/"
response_nasa = requests.get(url_nasa)
soup_nasa = BeautifulSoup(response_nasa.text, 'html.parser')

In [ ]:
news_title = soup_nasa.find('div', class_='content_title').text.strip()
news_p = soup_nasa.find('div', class_='rollover_description_inner').text.strip()
print('News title: ', news_title)
print('News paragraph: ', news_p)

### Use splinter to navigate page for featured image

In [ ]:
executable_path = {'executable_path': r'C:\bin\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url_jpl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_jpl)
browser.click_link_by_partial_text('FULL IMAGE')
browser.click_link_by_partial_href('/spaceimages/details')

In [ ]:
html_jpl = browser.html
soup_jpl = BeautifulSoup(html_jpl, 'html.parser')

In [ ]:
featured_image_url = 'https://www.jpl.nasa.gov/' + soup_jpl.find('img', class_="main_image")['src']

### Scrape Mars Weather twitter

In [ ]:
url_twtr = 'https://twitter.com/marswxreport?lang=en'
response_twtr = requests.get(url_twtr)
soup_twtr = BeautifulSoup(response_twtr.text, 'html.parser')

In [ ]:
mars_weathers = soup_twtr.find_all('p', class_="TweetTextSize")
for tweet in mars_weathers:
    tweet.find('a').extract()
    if 'InSight sol' in tweet.text:
        mars_weather = tweet.text
        break

In [ ]:
print(f'Mars weather:\n{mars_weather}')

### Scrape Mars Facts

In [ ]:
url_fact = 'https://space-facts.com/mars/'
tables = pd.read_html(url_fact)

In [ ]:
df_mars = tables[0]
# df_mars.set_index([0])
# df_mars.to_html('Resources/mars_table.html', header=False, index=False)

In [ ]:
# Store table data into a list of two-element list, which will be looped through during html table rendering.
mars_facts = []
for i in range(len(df_mars[0])):
    mars_facts.append([df_mars[0][i], df_mars[1][i]])
mars_facts

### Scrape Mars Hemispheres

In [ ]:
url_hemi = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response_hemi = requests.get(url_hemi)

In [ ]:
soup_hemi = BeautifulSoup(response_hemi.text, 'html.parser')

In [ ]:
# find all the titles
results = soup_hemi.find_all('div', class_="description")
hemisphere_image_urls = []
info = {}

# open url_hemi with splinter
browser_hemi = Browser('chrome', **executable_path, headless=False)


for result in results:

    title = result.text
    
    browser_hemi.visit(url_hemi)
    browser_hemi.click_link_by_partial_text(title)
    html_img = browser_hemi.html
    soup_img = BeautifulSoup(html_img, 'html.parser')

    img_url = soup_img.find('li').a['href']
    
    info["title"] = title
    info["img_url"] = img_url
    
    hemisphere_image_urls.append(info)
    info = {}
    

In [ ]:
hemisphere_image_urls